In [62]:
import cv2
import numpy as np
import keras
from keras.models import load_model
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from tkinter import *
from tkinter import filedialog

In [63]:
def read_img(filepath):
    img = cv2.imread(filepath)
    return img

In [64]:
def preprocess(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    blur = cv2.GaussianBlur(gray, (3,3),1)
    threshold_img = cv2.adaptiveThreshold(blur,255,1,1,11,2)
    return threshold_img

In [65]:
def get_contours(sudoku):
    contour_1 = sudoku.copy()
    threshold = preprocess(sudoku)
    contour, hierarchy = cv2.findContours(threshold,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(contour_1, contour,-1,(0, 255, 0),3)
    plt.imshow(contour_1)
    return contour

In [66]:
def biggest_contour(contour):
    global biggest
    biggest = np.array([])
    max_area = 0
    for i in contour:
        area = cv2.contourArea(i)
        if area >50:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i , 0.02* peri, True)
            if area > max_area and len(approx) ==4:
                biggest = approx
                max_area = area
    return biggest ,max_area

In [67]:
def reframe(points):
    points = points.reshape((4, 2))
    points_new = np.zeros((4,1,2),dtype = np.int32)
    add = points.sum(1)
    points_new[0] = points[np.argmin(add)]
    points_new[3] = points[np.argmax(add)]
    diff = np.diff(points, axis =1)
    points_new[1] = points[np.argmin(diff)]
    points_new[2] = points[np.argmax(diff)]
    return points_new

In [68]:
def draw_biggest(sudoku):
    global biggest
    contour_2 = sudoku.copy()
    cv2.drawContours(contour_2, biggest,-1,(255, 0, 0),15)

In [69]:
def wrap(biggest, sudoku):
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0,0],[450,0],[0,450],[450,450]])
    matrix = cv2.getPerspectiveTransform(pts1,pts2)
    imagewrap = cv2.warpPerspective(sudoku,matrix,(450,450))
    imagewrap =cv2.cvtColor(imagewrap, cv2.COLOR_BGR2GRAY)
    return imagewrap

In [70]:
def splitcells(img):
    rows = np.vsplit(img,9)
    boxes = []
    for r in rows:
        cols = np.hsplit(r,9)
        for box in cols:
            box = cv2.resize(box, (28,28))
            boxes.append(box)
    return boxes

In [71]:
def prepare(img):
    img = np.float32(img)
    img = np.reshape(img, (1,28,28,1))
    img = img/255.0
    return img

In [72]:
def pre_process(image):
    blur = cv2.GaussianBlur(image, (3,3),0)
    threshold_img = cv2.adaptiveThreshold(blur,255,1,1,11,15)
    return threshold_img

In [73]:
def check_blank(img):
    val = img[9][9]
    for i in range(6, 22):
        for j in range(6, 22):
            if img[i][j]!=val:
                return False
    return True

In [74]:
def get_matrix(sudoku_cell):
    model = load_model("Models/DigitClassifier.h5")
    M = []
    for img in sudoku_cell:
        img = pre_process(img)
        I = prepare(img)
        if check_blank(img):
            M.append(0)
        else:
            prediction = np.argmax(model.predict(I)[0])
            M.append(prediction)
    Sudoku_Matrix = []
    rownumber = -1
    for i in range(9):
        L = []
        rownumber+=1
        for j in range(9):
            L.append(M[j+rownumber*(9)])
        Sudoku_Matrix.append(L)
    return Sudoku_Matrix

In [75]:
def process(filepath):
    img = read_img(filepath)
    contour = get_contours(img)
    biggest ,max_area = biggest_contour(contour)
    if biggest.size!=0:
        biggest = reframe(biggest)
    draw_biggest(img)
    sudoku = wrap(biggest,img)
    plt.imshow(sudoku)
    sudoku_cells = splitcells(sudoku)
    sudoku_matrix = get_matrix(sudoku_cells)
    sudoku = cv2.cvtColor(sudoku, cv2.COLOR_GRAY2RGB)
    return sudoku, sudoku_matrix

In [84]:
def display_solution(img, numbers, color=(0, 255, 0)):
    W = int(img.shape[1]/9)
    H = int(img.shape[0]/9)
    for x in range(0, 9):
        for y in range(0, 9):
            if numbers[x][y]!=0:
                cv2.putText(img, str(numbers[x][y]), (y*W+10, (x+1)*H-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, color, 2, cv2.LINE_AA)
    return img